# Neural Turing Machines (NTM)

Alex Graves, Greg Wayne, Ivo Danihelka
$$
\newcommand{\memory}{\mathbf{M}}
\newcommand{\read}{\mathbf{r}}
\newcommand{\erase}{\mathbf{e}}
\newcommand{\add}{\mathbf{a}}
\newcommand{\weight}{\mathbf{w}}
\newcommand{\key}{\mathbf{k}}
\newcommand{\shift}{\mathbf{s}}
\newcommand{\Shift}{\mathbf{S}}
$$


##`$ finger shawntan`
(by the way, `finger` is totally the correct command to use here, `whoami` doesn't make sense at all.)
```
Login name: shawntan                    In real life: Tan Jing Shan,Shawn

Occupation: Research Assistant @ SoC (Speech Recognition Group)

- Graduated from NUS
- Worked at Semantics3

Research Interests: 
- Neural Networks
- Machine Learning
- Natural Language Processing
- Artificial Intelligence
```
### PLEASE interrupt me if I'm not clear. It helps.

## Here's the plan

1. Motivation behind doing this.
2. The Model.
3. Where things (have gone and) should go from here.

## Motivation

Let's look at the tasks they'll make the NTM perform...
1. **Copy**
   Give a variable input length of binary vectors. Once I'm done, regurgitate everything I gave you.
   
2. **Repeat copy**
   Give a variable input length of binary vectors. Once I'm done, I'll give you a number N. Regurgitate everything I gave you N times.   
   
3. **Associative Recall**
   Give a sequence of items. When shown one item from the sequence, show the next item in line.
   
4. **Dynamic N-Grams**
   "Learning to learn" an N-gram distribution estimator. Give you binary sequence, try to predict the next in line.
   
5. **Priority Sort**
   Give a sequence of priority-symbol pairs, output is symbol sequence sorted by priority value.

### Recurrent Neural Networks (GRUs, LSTMs, they're all the same..)

GRUs and LSTMs can perform these tasks, so why bother?

My hunch?
1. **Better generalisation** More unlikely for the network to just memorise familiar sequences from the training data.
2. **Visualisation, introspection** Being able to visualise what the network is doing by limiting the actions it can take.

## The Model

Just think of it like this...
<img src="images/controller.png"/>
* Controller: Multi-layer perceptron (MLP) or LSTM
* Data structure: 128 x 20 array (128 is the number of 'slots')
* Input: Depends on task

### The Read and Write heads

The paper mentions these "heads" often. But never really says how many there are and what they do.
1. I've assumed that there are separate read and write heads.
2. The heads decide where the acton (read or write) will occur in memory.

### How do we read from memory?
We need to read one row from the array.
1. Should be able to figure out on its own where to read from.
2. Should be able to take derivatives through the process.

Think about how we use a one-hot vector to select a row from the matrix...

$$
\left[\begin{matrix}
0 \\
\vdots \\
1 \\
\vdots\\
0
\end{matrix} \right]^{\top}
\underbrace{\left[\begin{matrix}
r_{0,0} & \cdots & r_{0,d}\\
&\vdots& \\
r_{i,0} & \cdots & r_{i,d} \\
&\vdots&\\
r_{|V|,0} & \cdots & r_{|V|,d}
\end{matrix} \right]}_{C}
= \left[r_{i,0} ~~ \cdots ~~ r_{i,d}\right] = C_i
$$

Their approach? Construct a vector that sums to 1, and use that to select the correct row by multiplying. This is the vector that is repeatedly referred to as $\mathbf{w}_t$ or $w_t(i)$.


### How do we write to memory?

The idea is still the same, but now we have 3 things, a write head $\weight_t$, an erase vector $\erase_t$, and an add vector $\add_t$. 

The process is then to,
1. "Softly" erase the relevant stuff from the row of memory of the previous time step
   $$\hat{\memory_t} (i) \leftarrow \memory_{t-1}(i)\left[1 - w_t(i)\erase_t\right]$$
2. "Softly" add the new data to the row of the modified memory
   $$\memory_t(i) \leftarrow \hat{\memory}_t(i) + w_t(i) \add_t$$

#### Key points:
1. Notice that in order for backpropagation through time to work for these "external memory" structures, the state of the memory at _every_ time step has to be kept.
2. This method of using the "expected value" or "soft" read and writes, is referred to as _attention_ in current deep learning literature.


### A word about the "heads"

How to generate these $\weight_t$ vectors?

1. **Content-based**
   * Have a way to lookup memory by content (e.g. nearest neighbour, cosine simiilarity...)
   * Similar to a hash-table type of lookup
   
2. **Location-based**
   * Perform a shift based on $\weight_{t-1}$ 
   * Similar to a increment in pointer or an increment to an array index.
   
And it also needs to decide between using either option.

### Constructing heads

<img src="https://blog.wtf.sg/wp-content/uploads/2014/10/Screenshot-from-2014-10-30-125051.png"/>



* $\key$: For looking up values within memory (via use of a similarity function)
* $\beta$: "Sharpening" factor for lookup
* $g$: For interpolating/switching between content lookup and location lookup
* $\shift$: Content lookup --- Shifting a step from $\weight_{t-1}$
* $\gamma$: Attention "sharpener"
$$\begin{align} 
\key &= \hat{\key} &\\ 
\beta &= \log\left(e^{\hat{\beta}} + 1 \right)  &\Longrightarrow &\beta > 0 \\ 
g &= \sigma\left(\hat{g}\right) &\Longrightarrow & g \in (0,1) \\ 
(\shift)_i &= \frac{\exp((\hat{\shift})_i)}{\sum_j \exp((\hat{\shift})_j)} &\Longrightarrow 
& (\shift)_i \in (0,1),\sum_i (\shift)_i = 1 \\ 
\gamma &= \log\left(e^{\hat{\gamma}} + 1 \right) + 1 &\Longrightarrow & \gamma \geq 1 \\ 
\end{align}$$

### So... what does the controller look like?

<img src="images/controller_nndiag.png"/>